<!-- doe2 -->

# DOE 2


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [ ]:
def gaussian_function(x, y, pars=None, add_noise=False):
    # to be changed: genereate all data here
    np.random.seed(2)
    if not pars:
        x0 = np.random.randn(1)[0]+0.5
        y0 = x0+0.5
        fwhm = np.random.normal(loc=1.5,scale=3, size=100).mean()
    else:
        x0, y0, fwhm = pars
        
    func = 10*np.exp(-4*np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)
    
    if add_noise:
        func = add_noise_func(func)
    
    return func, x0, y0, fwhm

def add_noise_func(input_data, mean_noise=0):
    # incomplete: refactor code to generate data + noise in one function
    avg_data = np.mean(input_data)
    noise_ = np.random.normal(mean_noise, np.sqrt(avg_data), 1)[0]
    output_data = input_data + noise_
    
    return output_data

def add_final_points(temp_df):
        mean_x, mean_y, range_x, range_y = normal_to_coded_var(temp_df, output_pars = True) 
        
        temp_df.loc[len(temp_df)] = [mean_x+range_x,mean_y]
        temp_df.loc[len(temp_df)] = [mean_x-range_x,mean_y]
        temp_df.loc[len(temp_df)] = [mean_x,mean_y+range_y]
        temp_df.loc[len(temp_df)] = [mean_x,mean_y-range_y]
        
        return temp_df
    
def output_model(x_d1, y_d1, model=None, pars=None, cnt_point=True, final_stage=False) -> pd.DataFrame:
    # inputs 2 arrays of coordinates, outputs a full factorial
    temp_df = pd.DataFrame([(i,j) for i in x_d1 for j in y_d1], columns=['x','y'])

    if cnt_point:
        temp_df.loc[len(temp_df)] = [temp_df['x'].mean(),temp_df['y'].mean()]
        
    if final_stage:
        temp_df = add_final_points(temp_df)
        
    temp_df = normal_to_coded_var(temp_df)    
    temp_df['outcome'] = temp_df.apply(lambda x: gaussian_function(x['x'], x['y'], pars)[0], axis=1)
    
    if model:
        temp_df['predict'] = model.predict(temp_df[['x','y']])
        temp_df['mape'] = 100*(temp_df['outcome'] - temp_df['predict'])/temp_df['outcome']
        
    return temp_df

def normal_to_coded_var(input_df, output_pars = False):
    # mapping between original and coded variables 
    temp_df = input_df.copy()
    mean_x = temp_df['x'].mean()
    mean_y = temp_df['y'].mean()
    range_x = temp_df['x'].max()-temp_df['x'].min()
    range_y = temp_df['y'].max()-temp_df['y'].min()
    
    temp_df['X'] = (temp_df['x']-mean_x)/(range_x/2)
    temp_df['Y'] = (temp_df['y']-mean_y)/(range_y/2)
    
    if output_pars:
        return mean_x, mean_y, range_x, range_y
    
    return temp_df

def coded_to_normal(pars, x_coded, y_coded):
    # mapping between coded and normal variables
    mean_x, mean_y, range_x, range_y = pars
    
    real_x = x_coded*(range_x/2) + mean_x
    real_y = y_coded*(range_y/2) + mean_y
    
    return real_x, real_y
    
def plot_contour_fromdf(test_df, fill_value=0, model=None, levels=10, plot_normal_vars=True):
    # surface plots
    # if there's no model try to plot a contour plot given the dataframe
    # otherwise use the model to predict values from min to max value of each dimension
    # x, y = 'cX','cY'
    x, y = 'x','y'
    
    
    if not model:
        hdf = test_df.groupby(input_vars).mean()
        hdfreset = hdf.reset_index()
        hdfreset.columns = [x, y, 'outcome']
        hdfpivot= hdfreset.pivot(x, y).fillna(0)

        X=hdfpivot.columns.levels[1].values
        Y=hdfpivot.index.values
        Xi,Yi = np.meshgrid(X, Y)
        Z=hdfpivot.values
    else:
        X = np.linspace(test_df[x].min(),test_df[x].max(),levels)
        Y = np.linspace(test_df[y].min(),test_df[y].max(),levels)
        Xi,Yi = np.meshgrid(X, Y)
        Z = model.predict(np.c_[Xi.ravel(),Yi.ravel()]).reshape(len(X),len(Y))
        
    if plot_normal_vars:
        temp_df = pd.DataFrame()
        temp_df[x] = Xi.ravel()
        temp_df[y] = Yi.ravel()
        Z = model.predict(temp_df).reshape(len(X),len(Y))
        mean_x, mean_y, range_x, range_y = normal_to_coded_var(test_df, output_pars = True)
        temp_df['x'] = temp_df[x]*(range_x/2) + mean_x
        temp_df['y'] = temp_df[y]*(range_y/2) + mean_y
        X = np.linspace(test_df['x'].min(),test_df['x'].max(),levels)
        Y = np.linspace(test_df['y'].min(),test_df['y'].max(),levels)
        
        Xi,Yi = np.meshgrid(X, Y)
    else:
        mean_x, mean_y = 0, 0
        
    # gradients given Z to plot quiverplot    
    v, u = np.gradient(Z, 0.1, 0.1)
    plt.contourf(Xi, Yi, Z, alpha=0.7)
    plt.colorbar()
    plt.quiver(Xi,Yi,u,v);
    
def linear_pipe_fit(degree=2, interaction=True, dataframe=pd.DataFrame(), input_vars=['x','y']):
    
    lr = LinearRegression()
    pol = PolynomialFeatures(degree=degree, interaction_only=interaction, include_bias=False)
    pipe = Pipeline([('pol',pol), ('lr', lr)])
    pipe.fit(dataframe[input_vars], dataframe['outcome'])
    coefs = pipe['lr'].coef_
    intercept = pipe['lr'].intercept_

    return pipe

def next_step(model=None, increment_y = 1, step_x = 0.125, step_y = 0.125, dataframe=pd.DataFrame()):
    
    mean_x, mean_y, range_x, range_y = normal_to_coded_var(dataframe, output_pars = True)
    coef_x = model['lr'].coef_[0]
    coef_y = model['lr'].coef_[1]
    ratio = coef_x/coef_y
    
    print(f'coef x {coef_x:.2f} and coef y {coef_y:.2f}')
    
    increment_x = np.abs(ratio)*increment_y*np.sign(coef_x)
    increment_y = increment_y*np.sign(coef_y)
    
    print(f'increment x {increment_x:.2f} and increment y {increment_y:.2f} with ratio {ratio:.2f}')
    
    base_x = increment_x*(range_x/2)+mean_x
    base_y = increment_y*(range_y/2)+mean_y
    
    print(f'new x {base_x:.2f} and new y {base_y:.2f}')
                
    next_x = np.round(np.array([base_x-step_x, base_x+step_x]),2)
    next_y = np.round(np.array([base_y-step_y, base_y+step_y]),2)
    
    return next_x, next_y

def final_step(model=None, pars=None, dataframe=pd.DataFrame(), final_stage=True):
    # do the last step with an increased precision
    next_x, next_y = next_step(model=model, dataframe=dataframe)
    test_df = output_model(next_x, next_y, pars=pars, cnt_point=True, final_stage=final_stage)
    return test_df

In [ ]:
x = np.round(np.linspace(-2,2,101),2)
y = np.round(np.linspace(-2,2,101),2)
X, Y = np.meshgrid(x,y)
Z, x0, y0, fwhm = gaussian_function(X,Y,add_noise=True)
plt.contourf(x,y,Z, levels=10, vmin=0)
plt.colorbar()

## first full factorial


In [ ]:
x_d1 = np.array([0.5,1])
y_d1 = np.array([-0.5,-1])
first_doe = output_model(x_d1, y_d1, pars=(x0, y0, fwhm), cnt_point=False)
first_doe

In [ ]:
plt.contourf(x,y,Z, levels=10, vmin=0)
plt.colorbar()
plt.scatter(first_doe.x,first_doe.y, c='r')

In [ ]:
pipe1 = linear_pipe_fit(degree=2, interaction=True, dataframe=first_doe, input_vars=['x','y'])
plot_contour_fromdf(first_doe, fill_value=0, model = pipe1, plot_normal_vars=True)

## Second Step


In [ ]:
x_d2, y_d2 = next_step(model=pipe1, increment_y = 2, dataframe=first_doe)
print(x_d2, y_d2)

In [ ]:
test_df2 = output_model(x_d2, y_d2, model = pipe1, pars=(x0, y0, fwhm), cnt_point=True)
pipe2 = linear_pipe_fit(degree=1, interaction=True, dataframe=test_df2, input_vars=['x','y'])
plot_contour_fromdf(test_df2, fill_value=0, model = pipe2)
x_d3, y_d3 = next_step(model=pipe2, increment_y = 2, dataframe=test_df2)
print(x_d3, y_d3)

In [ ]:
test_df2

In [ ]:
plt.contourf(x,y,Z, levels=10, vmin=0)
plt.colorbar()
plt.scatter(first_doe.x,first_doe.y, c='r')
plt.scatter(test_df2.x,test_df2.y, c='b')

## Third Step


In [ ]:
test_df3 = output_model(x_d3, y_d3, pipe2, pars=(x0, y0, fwhm), cnt_point=False)
pipe3 = linear_pipe_fit(degree=1, interaction=True, dataframe=test_df3, input_vars=['x','y'])
plot_contour_fromdf(test_df3, fill_value=0, model = pipe3)
x_d4, y_d4 = next_step(model=pipe3, increment_y = 2, dataframe=test_df3)
print(x_d4, y_d4)

In [ ]:
test_df3

In [ ]:
plt.contourf(x,y,Z, levels=10, vmin=0)
plt.colorbar()
plt.scatter(first_doe.x,first_doe.y, c='r')
plt.scatter(test_df2.x,test_df2.y, c='b')
plt.scatter(test_df3.x,test_df3.y, c='k')

# Final Implementation


In [ ]:
steps_df = pd.DataFrame()
x_t = np.array([0.5,1])
y_t = np.array([-0.5,-1])

for i in range(10):
    test_df = output_model(x_t, y_t, pars=(x0, y0, fwhm), cnt_point=False)
    test_df['iteration'] = i
    pipe1 = linear_pipe_fit(degree=1, interaction=True, dataframe=test_df, input_vars=['x','y'])
    x_t, y_t = next_step(pipe1, increment_y = 2, dataframe=test_df)
    #plot_contour_fromdf(test_df, fill_value=0, model = pipe1, plot_normal_vars=True)
    steps_df = steps_df.append(test_df)
    max_outcome = test_df['outcome'].max()
    
    if max_outcome < steps_df['outcome'].max():
        final_ = final_step(model=pipe1, pars=(x0, y0, fwhm), dataframe=test_df)
        final_['iteration'] = i
        pipe_last = linear_pipe_fit(degree=2, interaction=True, dataframe=final_, input_vars=['x','y'])
        steps_df = steps_df.append(final_)
        print(f'maximum found at iteration {i}')
        break
    print(x_t, y_t)

In [ ]:
plt.contourf(x,y,Z)
plt.colorbar()
sns.scatterplot(x='x',y='y',data=steps_df, hue='iteration', palette='coolwarm', legend='full', edgecolor='k')

In [ ]:
steps_df[steps_df.iteration==6]

In [ ]:
pol = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
lr = LinearRegression()
pipe_all = Pipeline([('pol',pol), ('lr', lr)])
pipe_all.fit(steps_df[['x','y']], steps_df['outcome'])
pipe_all['lr'].coef_

In [ ]:
plot_contour_fromdf(steps_df, fill_value=0, model = pipe_all, levels=30)

In [ ]:
pipe_all.score(steps_df[['x','y']], steps_df['outcome'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(max_depth=8)
rfr.fit(steps_df[['x','y']], steps_df['outcome'])
plot_contour_fromdf(steps_df, fill_value=0, model = rfr, levels=30)